In [1]:
# %cd drive/My\ Drive/Colab\ Notebooks

In [2]:
# train_networks: Training CNN to be used by the main program

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2
import os
import imutils
from processing import *

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import random_split, Dataset, DataLoader

from skimage.segmentation import watershed
from skimage.morphology import disk
from skimage.feature import peak_local_max
from skimage.filters import meijering

from scipy import ndimage as ndi

BATCH_SIZE = 2
IMAGES_LIMIT = 5000

DROP_IN = 1000

def plot_two_images(imgL, imgR, titleL, titleR):
    f = plt.figure()
    f.add_subplot(1,2, 1)
    plt.imshow(imgL, cmap='gray')
    plt.title(titleL)
    f.add_subplot(1,2, 2)
    plt.imshow(imgR, cmap='gray')
    plt.title(titleR)
    plt.show(block=True)
    

# def load_data(dataset):
#     data = []
#     paths = [os.path.join(dataset, '01'), os.path.join(dataset, '02')]
#     for path in paths:
#         mask_path = path + '_ST'
#         mask_path = os.path.join(mask_path, 'SEG')
#         for f in os.listdir(mask_path):
#             if not f.endswith(".tif"):
#                 continue
#             image = cv2.imread(os.path.join(path, f.replace('man_seg', 't')), cv2.IMREAD_GRAYSCALE)
#             image = equalize_clahe(image).astype(np.float32)
#             mask = cv2.imread(os.path.join(mask_path, f), cv2.IMREAD_UNCHANGED)
#             print("   Loaded " + os.path.join(mask_path, f) + ", " + os.path.join(path, f.replace('man_seg', 't')))
            
#             # Generate the Cell Mask and Markers from the Mask
#             cell_mask = (mask > 0).astype(np.uint8)
#             markers = (get_markers(mask) > 0).astype(np.uint8)
#             weight_map = get_weight_map(markers)
            
#             # Pack the data for the DataLoader
#             target = (cell_mask, markers, weight_map)
#             data.append((np.array([image]), target))

#     train_size = int(0.8 * len(data))
#     test_size = len(data) - train_size
#     train_data, test_data = random_split(data, [train_size, test_size])
#     trainLoader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
#     testLoader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)
#     return trainLoader, testLoader

def load_data(dataset):
    data = []
    path = os.path.join(dataset, "originals")
    clahe_path = path.replace("originals", "clahes")
    mask_path = path.replace("originals", "masks")
    markers_path = path.replace("originals", "markers")
    wm_path = path.replace("originals", "weight_maps")
    
    
    count = 0
    hid_cnt = 0
    for f in os.listdir(path):
        # Limits dataset size for RAM compatibility
        if count >= IMAGES_LIMIT:
            break
            
        # Limits dataset size for RAM compatibility
        hid_cnt += 1
        if hid_cnt % DROP_IN == 0:
            continue
        
        if not f.endswith(".npy"):
            continue
#         image = cv2.imread(os.path.join(path, f), cv2.IMREAD_GRAYSCALE)
#         clahe = cv2.imread(os.path.join(clahe_path, f), cv2.IMREAD_GRAYSCALE).astype(np.float32)
#         cell_mask = cv2.imread(os.path.join(mask_path, f), cv2.IMREAD_UNCHANGED)
#         markers = cv2.imread(os.path.join(markers_path, f), cv2.IMREAD_UNCHANGED)
#         weight_map = cv2.imread(os.path.join(wm_path, f), cv2.IMREAD_UNCHANGED)
    
        count += 1
    
#         image = np.load(os.path.join(path, f))
        clahe = np.load(os.path.join(clahe_path, f))
        cell_mask = np.load(os.path.join(mask_path, f))
        markers = np.load(os.path.join(markers_path, f))
        weight_map = np.load(os.path.join(wm_path, f))
        print("   Loaded " + os.path.join(mask_path, f) + ", " + os.path.join(path, f.replace('mask', '')))

        # Pack the data for the DataLoader
        target = (cell_mask, markers, weight_map)
        data.append((np.array([clahe]), target))
    
#     # For RAM
#     keep_size = int(KEEP_RATIO * len(data))
#     data, _ = random_split(data, [keep_size, len(data) - keep_size])
    
    train_size = int(0.8 * len(data))
    test_size = len(data) - train_size
    train_data, test_data = random_split(data, [train_size, test_size])
    trainLoader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
    testLoader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)
    return trainLoader, testLoader

In [3]:
# Class for creating the CNN
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv5 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv6 = nn.Conv2d(128, 128, 3, padding=1)
        self.conv7 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv8 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv9 = nn.Conv2d(256, 512, 3, padding=1)
        self.conv10 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv11 = nn.Conv2d(768, 256, 3, padding=1)
        self.conv12 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv13 = nn.Conv2d(384, 128, 3, padding=1)
        self.conv14 = nn.Conv2d(128, 128, 3, padding=1)
        self.conv15 = nn.Conv2d(192, 64, 3, padding=1)
        self.conv16 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv17 = nn.Conv2d(96, 32, 3, padding=1)
        self.conv18 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv_out = nn.Conv2d(32, 2, 1)
        
    def forward(self, x):
        """
        Forward pass through the network
        """
        x = F.relu(self.conv1(x))
        contraction_32 = F.relu(self.conv2(x))
        
        x = F.max_pool2d(contraction_32, kernel_size=2)
        x = F.relu(self.conv3(x))
        contraction_64 = F.relu(self.conv4(x))
        
        x = F.max_pool2d(contraction_64, kernel_size=2)
        x = F.relu(self.conv5(x))
        contraction_128 = F.relu(self.conv6(x))
        
        x = F.max_pool2d(contraction_128, kernel_size=2)
        x = F.relu(self.conv7(x))
        contraction_256 = F.relu(self.conv8(x))
        
        x = F.max_pool2d(contraction_256, kernel_size=2)
        x = F.relu(self.conv9(x))
        x = F.relu(self.conv10(x))
        
        x = F.interpolate(x, scale_factor=2, mode='bilinear')
        x = torch.cat((contraction_256, x), dim=1)
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        
        x = F.interpolate(x, scale_factor=2, mode='bilinear')
        x = torch.cat((contraction_128, x), dim=1)
        x = F.relu(self.conv13(x))
        x = F.relu(self.conv14(x))
        
        x = F.interpolate(x, scale_factor=2, mode='bilinear')
        x = torch.cat((contraction_64, x), dim=1)
        x = F.relu(self.conv15(x))
        x = F.relu(self.conv16(x))
        
        x = F.interpolate(x, scale_factor=2, mode='bilinear')
        x = torch.cat((contraction_32, x), dim=1)
        x = F.relu(self.conv17(x))
        x = F.relu(self.conv18(x))
        
        x = self.conv_out(x)
        output = F.sigmoid(x)
        return output

def weighted_mean_sq_error(inputs, targets_m, targets_c, weights):

#     Weighted Mean Squared Error Loss takes in a weight map
#     and computes loss based on both outputs
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print('.', end='')
    
#     print(inputs.shape, targets_m.shape)
    
    inputs = inputs.to(device)
    targets = [targets_m.to(device), targets_c.to(device)]
    weights = weights.to(device)
    loss = torch.zeros(inputs.shape[0])

    # Calculate loss for each sample in the batch
    for sample in range(inputs.shape[0]):
#         print("Sample", sample+1)
        sample_loss, total_weight = 0.0, 0.0
        
        pred_markers = inputs[sample][0]
        pred_cmask = inputs[sample][1]
        
#         pimg(pred_markers.cpu().detach().numpy())
#         pimg(pred_cmasks.cpu().detach().numpy())
#         pimg(weights.cpu().detach().numpy())
        
        exp_markers = targets[0][sample]
        exp_cmask = targets[1][sample]
        
        numerator = weights[sample] * ( (pred_markers-exp_markers)**2 + (pred_cmask-exp_cmask)**2 )
        
        numerator = torch.sum(numerator)
        
        denominator = torch.sum(weights[sample])
#         print(np.unique(exp_markers), np.unique(exp_cmasks), numerator, denominator)
        sample_loss = 0.5 * (numerator/denominator)
        
        loss[sample] = sample_loss

    return torch.mean(loss)

"""
def get_score(outputs, ground_truth):
    
    #Calculates Accuracy Score across the batch
    
    score = 0
    batch_size = outputs.shape[0]
    total = outputs.shape[1] * outputs.shape[2]
    for sample in range(batch_size):
        num_correct = torch.sum(outputs[sample] == ground_truth[sample]).item()
        score += float(num_correct) / total

    return score / batch_size
"""


def main():
    """
    Train 2 networks for predicting markers and the cell mask respectively
    Set trains on data from "Sequence 1 Masks" and "Sequence 2 Masks"
    and save the models
    """    
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Using device: " + str(device))
    
    # Net predicts the markers and cells
    net = Network().to(device)
    
    # Calculate loss with Weighted MSE
    criterion = weighted_mean_sq_error
    
    # Optimising using Adam algorithm
    optimiser = optim.Adam(net.parameters(), lr=0.001)
    
    min_loss = 9999999999
    file_count_c = 0
    file_count_m = 0
    
    # Iterate over a number of epochs on the data
    for epoch in range(100):
        for i, batch in enumerate(trainLoader, 0):
            x = batch[0].to(device)
            target = batch[1]
            cell_masks, markers = target[0].to(device), target[1].to(device) # Unpack target data
            weight_map = target[2].to(device)

            # Clear gradients from last step
            optimiser.zero_grad()
            
            # Predict the markers from the image
            output = net(x)
            # loss_m = criterion(output_m, markers.long())
            loss = criterion(output, markers.float(), cell_masks.float(), weight_map)
            loss.backward()
            optimiser.step()

            if i == 0 or (i + 1) % 960 == 0:
                print(f"Epoch: {epoch+1}, Batch: {i + 1}")
                print(f"Loss: {loss.item():.2f}")
                
                plt.imshow(x[0][0].cpu(), cmap='gray')
                plt.title("Input")
                plt.show()

                # Get the predicted Cell Mask and Markers for one of the images
                pred_m = ( (output[0][0] > 0.5).int() ).cpu()
                pred_c = ( (output[0][1] > 0.5).int() ).cpu()
                
                # Compare predicted to true images
                plot_two_images(pred_c.cpu(), cell_masks[0].cpu(), "Predicted Cell Mask", "True Cell Mask")
                plot_two_images(pred_m.cpu(), markers[0].cpu(), "Predicted Markers", "True Markers")


        # Test on the evaluation set
        print("\n--- Evaluation ---")
        net.eval()
        with torch.no_grad():
            running_loss = 0.0
            for i, batch in enumerate(testLoader):
                x = batch[0].to(device)
                target = batch[1]
                cell_masks, markers = target[0].to(device), target[1].to(device) # Unpack target data
                weight_map = target[2].to(device)

                output = net(x)
                
                pred_m = (output[:,0] > 0.5).int()
                pred_c = (output[:,1] > 0.5).int()

                running_loss += weighted_mean_sq_error(output, markers.float(), cell_masks.float(), weight_map)

                if i == 0:
                    plt.imshow(x[0][0].cpu(), cmap='gray')
                    plt.title("Input")
                    plt.show()

                    # Compare predicted to true images
                    plot_two_images(pred_c[0].cpu(), cell_masks[0].cpu(), "Predicted Cell Mask", "True Cell Mask")
                    plot_two_images(pred_m[0].cpu(), markers[0].cpu(), "Predicted Markers", "True Markers")

            loss = running_loss / len(testLoader)
            
            if loss < min_loss:
                torch.save(net.state_dict(), "./CNN_min_loss_{}.pth".format(file_count_c))
                min_loss = loss
                file_count_c += 1
                file_count_c %= 20
            
            print(f"EPOCH {epoch+1} LOSS: {loss:.3f}\nMin Loss: {min_loss:.3f}\n\n")
        net.train()

    torch.save(net.state_dict(), "./CNN_v1.pth")
    print("Saved models. Min Loss : ", str(min_loss))

In [4]:
print("Loading Data...")
trainLoader, testLoader = load_data('Fluo-4_cache')
print("Finished.")

Loading Data...
   Loaded Fluo-4_cache\masks\t0.npy, Fluo-4_cache\originals\t0.npy
   Loaded Fluo-4_cache\masks\t1.npy, Fluo-4_cache\originals\t1.npy
   Loaded Fluo-4_cache\masks\t10.npy, Fluo-4_cache\originals\t10.npy
   Loaded Fluo-4_cache\masks\t100.npy, Fluo-4_cache\originals\t100.npy
   Loaded Fluo-4_cache\masks\t1000.npy, Fluo-4_cache\originals\t1000.npy
   Loaded Fluo-4_cache\masks\t1001.npy, Fluo-4_cache\originals\t1001.npy
   Loaded Fluo-4_cache\masks\t1002.npy, Fluo-4_cache\originals\t1002.npy
   Loaded Fluo-4_cache\masks\t1003.npy, Fluo-4_cache\originals\t1003.npy
   Loaded Fluo-4_cache\masks\t1004.npy, Fluo-4_cache\originals\t1004.npy
   Loaded Fluo-4_cache\masks\t1005.npy, Fluo-4_cache\originals\t1005.npy
   Loaded Fluo-4_cache\masks\t1006.npy, Fluo-4_cache\originals\t1006.npy
   Loaded Fluo-4_cache\masks\t1007.npy, Fluo-4_cache\originals\t1007.npy
   Loaded Fluo-4_cache\masks\t1008.npy, Fluo-4_cache\originals\t1008.npy
   Loaded Fluo-4_cache\masks\t1009.npy, Fluo-4_cache\

   Loaded Fluo-4_cache\masks\t11.npy, Fluo-4_cache\originals\t11.npy
   Loaded Fluo-4_cache\masks\t110.npy, Fluo-4_cache\originals\t110.npy
   Loaded Fluo-4_cache\masks\t1100.npy, Fluo-4_cache\originals\t1100.npy
   Loaded Fluo-4_cache\masks\t1101.npy, Fluo-4_cache\originals\t1101.npy
   Loaded Fluo-4_cache\masks\t1102.npy, Fluo-4_cache\originals\t1102.npy
   Loaded Fluo-4_cache\masks\t1103.npy, Fluo-4_cache\originals\t1103.npy
   Loaded Fluo-4_cache\masks\t1104.npy, Fluo-4_cache\originals\t1104.npy
   Loaded Fluo-4_cache\masks\t1105.npy, Fluo-4_cache\originals\t1105.npy
   Loaded Fluo-4_cache\masks\t1106.npy, Fluo-4_cache\originals\t1106.npy
   Loaded Fluo-4_cache\masks\t1107.npy, Fluo-4_cache\originals\t1107.npy
   Loaded Fluo-4_cache\masks\t1108.npy, Fluo-4_cache\originals\t1108.npy
   Loaded Fluo-4_cache\masks\t1109.npy, Fluo-4_cache\originals\t1109.npy
   Loaded Fluo-4_cache\masks\t111.npy, Fluo-4_cache\originals\t111.npy
   Loaded Fluo-4_cache\masks\t1110.npy, Fluo-4_cache\origin

   Loaded Fluo-4_cache\masks\t1202.npy, Fluo-4_cache\originals\t1202.npy
   Loaded Fluo-4_cache\masks\t1203.npy, Fluo-4_cache\originals\t1203.npy
   Loaded Fluo-4_cache\masks\t1204.npy, Fluo-4_cache\originals\t1204.npy
   Loaded Fluo-4_cache\masks\t1205.npy, Fluo-4_cache\originals\t1205.npy
   Loaded Fluo-4_cache\masks\t1206.npy, Fluo-4_cache\originals\t1206.npy
   Loaded Fluo-4_cache\masks\t1207.npy, Fluo-4_cache\originals\t1207.npy
   Loaded Fluo-4_cache\masks\t1208.npy, Fluo-4_cache\originals\t1208.npy
   Loaded Fluo-4_cache\masks\t1209.npy, Fluo-4_cache\originals\t1209.npy
   Loaded Fluo-4_cache\masks\t121.npy, Fluo-4_cache\originals\t121.npy
   Loaded Fluo-4_cache\masks\t1210.npy, Fluo-4_cache\originals\t1210.npy
   Loaded Fluo-4_cache\masks\t1211.npy, Fluo-4_cache\originals\t1211.npy
   Loaded Fluo-4_cache\masks\t1212.npy, Fluo-4_cache\originals\t1212.npy
   Loaded Fluo-4_cache\masks\t1213.npy, Fluo-4_cache\originals\t1213.npy
   Loaded Fluo-4_cache\masks\t1214.npy, Fluo-4_cache\

   Loaded Fluo-4_cache\masks\t1304.npy, Fluo-4_cache\originals\t1304.npy
   Loaded Fluo-4_cache\masks\t1305.npy, Fluo-4_cache\originals\t1305.npy
   Loaded Fluo-4_cache\masks\t1306.npy, Fluo-4_cache\originals\t1306.npy
   Loaded Fluo-4_cache\masks\t1307.npy, Fluo-4_cache\originals\t1307.npy
   Loaded Fluo-4_cache\masks\t1308.npy, Fluo-4_cache\originals\t1308.npy
   Loaded Fluo-4_cache\masks\t1309.npy, Fluo-4_cache\originals\t1309.npy
   Loaded Fluo-4_cache\masks\t131.npy, Fluo-4_cache\originals\t131.npy
   Loaded Fluo-4_cache\masks\t1310.npy, Fluo-4_cache\originals\t1310.npy
   Loaded Fluo-4_cache\masks\t1311.npy, Fluo-4_cache\originals\t1311.npy
   Loaded Fluo-4_cache\masks\t1312.npy, Fluo-4_cache\originals\t1312.npy
   Loaded Fluo-4_cache\masks\t1313.npy, Fluo-4_cache\originals\t1313.npy
   Loaded Fluo-4_cache\masks\t1314.npy, Fluo-4_cache\originals\t1314.npy
   Loaded Fluo-4_cache\masks\t1315.npy, Fluo-4_cache\originals\t1315.npy
   Loaded Fluo-4_cache\masks\t1316.npy, Fluo-4_cache\

   Loaded Fluo-4_cache\masks\t1405.npy, Fluo-4_cache\originals\t1405.npy
   Loaded Fluo-4_cache\masks\t1406.npy, Fluo-4_cache\originals\t1406.npy
   Loaded Fluo-4_cache\masks\t1407.npy, Fluo-4_cache\originals\t1407.npy
   Loaded Fluo-4_cache\masks\t1408.npy, Fluo-4_cache\originals\t1408.npy
   Loaded Fluo-4_cache\masks\t1409.npy, Fluo-4_cache\originals\t1409.npy
   Loaded Fluo-4_cache\masks\t141.npy, Fluo-4_cache\originals\t141.npy
   Loaded Fluo-4_cache\masks\t1410.npy, Fluo-4_cache\originals\t1410.npy
   Loaded Fluo-4_cache\masks\t1411.npy, Fluo-4_cache\originals\t1411.npy
   Loaded Fluo-4_cache\masks\t1412.npy, Fluo-4_cache\originals\t1412.npy
   Loaded Fluo-4_cache\masks\t1413.npy, Fluo-4_cache\originals\t1413.npy
   Loaded Fluo-4_cache\masks\t1414.npy, Fluo-4_cache\originals\t1414.npy
   Loaded Fluo-4_cache\masks\t1415.npy, Fluo-4_cache\originals\t1415.npy
   Loaded Fluo-4_cache\masks\t1416.npy, Fluo-4_cache\originals\t1416.npy
   Loaded Fluo-4_cache\masks\t1417.npy, Fluo-4_cache\

...

   Loaded Fluo-4_cache\masks\t513.npy, Fluo-4_cache\originals\t513.npy
   Loaded Fluo-4_cache\masks\t514.npy, Fluo-4_cache\originals\t514.npy
   Loaded Fluo-4_cache\masks\t515.npy, Fluo-4_cache\originals\t515.npy
   Loaded Fluo-4_cache\masks\t516.npy, Fluo-4_cache\originals\t516.npy
   Loaded Fluo-4_cache\masks\t517.npy, Fluo-4_cache\originals\t517.npy
   Loaded Fluo-4_cache\masks\t518.npy, Fluo-4_cache\originals\t518.npy
   Loaded Fluo-4_cache\masks\t519.npy, Fluo-4_cache\originals\t519.npy
   Loaded Fluo-4_cache\masks\t52.npy, Fluo-4_cache\originals\t52.npy
   Loaded Fluo-4_cache\masks\t520.npy, Fluo-4_cache\originals\t520.npy
   Loaded Fluo-4_cache\masks\t521.npy, Fluo-4_cache\originals\t521.npy
   Loaded Fluo-4_cache\masks\t522.npy, Fluo-4_cache\originals\t522.npy
   Loaded Fluo-4_cache\masks\t523.npy, Fluo-4_cache\originals\t523.npy
   Loaded Fluo-4_cache\masks\t524.npy, Fluo-4_cache\originals\t524.npy
   Loaded Fluo-4_cache\masks\t525.npy, Fluo-4_cache\originals\t525.npy
   Loade

   Loaded Fluo-4_cache\masks\t619.npy, Fluo-4_cache\originals\t619.npy
   Loaded Fluo-4_cache\masks\t62.npy, Fluo-4_cache\originals\t62.npy
   Loaded Fluo-4_cache\masks\t620.npy, Fluo-4_cache\originals\t620.npy
   Loaded Fluo-4_cache\masks\t621.npy, Fluo-4_cache\originals\t621.npy
   Loaded Fluo-4_cache\masks\t622.npy, Fluo-4_cache\originals\t622.npy
   Loaded Fluo-4_cache\masks\t623.npy, Fluo-4_cache\originals\t623.npy
   Loaded Fluo-4_cache\masks\t624.npy, Fluo-4_cache\originals\t624.npy
   Loaded Fluo-4_cache\masks\t625.npy, Fluo-4_cache\originals\t625.npy
   Loaded Fluo-4_cache\masks\t626.npy, Fluo-4_cache\originals\t626.npy
   Loaded Fluo-4_cache\masks\t627.npy, Fluo-4_cache\originals\t627.npy
   Loaded Fluo-4_cache\masks\t628.npy, Fluo-4_cache\originals\t628.npy
   Loaded Fluo-4_cache\masks\t629.npy, Fluo-4_cache\originals\t629.npy
   Loaded Fluo-4_cache\masks\t63.npy, Fluo-4_cache\originals\t63.npy
   Loaded Fluo-4_cache\masks\t630.npy, Fluo-4_cache\originals\t630.npy
   Loaded 

   Loaded Fluo-4_cache\masks\t724.npy, Fluo-4_cache\originals\t724.npy
   Loaded Fluo-4_cache\masks\t725.npy, Fluo-4_cache\originals\t725.npy
   Loaded Fluo-4_cache\masks\t726.npy, Fluo-4_cache\originals\t726.npy
   Loaded Fluo-4_cache\masks\t727.npy, Fluo-4_cache\originals\t727.npy
   Loaded Fluo-4_cache\masks\t728.npy, Fluo-4_cache\originals\t728.npy
   Loaded Fluo-4_cache\masks\t729.npy, Fluo-4_cache\originals\t729.npy
   Loaded Fluo-4_cache\masks\t73.npy, Fluo-4_cache\originals\t73.npy
   Loaded Fluo-4_cache\masks\t730.npy, Fluo-4_cache\originals\t730.npy
   Loaded Fluo-4_cache\masks\t731.npy, Fluo-4_cache\originals\t731.npy
   Loaded Fluo-4_cache\masks\t732.npy, Fluo-4_cache\originals\t732.npy
   Loaded Fluo-4_cache\masks\t733.npy, Fluo-4_cache\originals\t733.npy
   Loaded Fluo-4_cache\masks\t734.npy, Fluo-4_cache\originals\t734.npy
   Loaded Fluo-4_cache\masks\t735.npy, Fluo-4_cache\originals\t735.npy
   Loaded Fluo-4_cache\masks\t736.npy, Fluo-4_cache\originals\t736.npy
   Loade

   Loaded Fluo-4_cache\masks\t829.npy, Fluo-4_cache\originals\t829.npy
   Loaded Fluo-4_cache\masks\t83.npy, Fluo-4_cache\originals\t83.npy
   Loaded Fluo-4_cache\masks\t830.npy, Fluo-4_cache\originals\t830.npy
   Loaded Fluo-4_cache\masks\t831.npy, Fluo-4_cache\originals\t831.npy
   Loaded Fluo-4_cache\masks\t832.npy, Fluo-4_cache\originals\t832.npy
   Loaded Fluo-4_cache\masks\t833.npy, Fluo-4_cache\originals\t833.npy
   Loaded Fluo-4_cache\masks\t834.npy, Fluo-4_cache\originals\t834.npy
   Loaded Fluo-4_cache\masks\t835.npy, Fluo-4_cache\originals\t835.npy
   Loaded Fluo-4_cache\masks\t836.npy, Fluo-4_cache\originals\t836.npy
   Loaded Fluo-4_cache\masks\t837.npy, Fluo-4_cache\originals\t837.npy
   Loaded Fluo-4_cache\masks\t838.npy, Fluo-4_cache\originals\t838.npy
   Loaded Fluo-4_cache\masks\t839.npy, Fluo-4_cache\originals\t839.npy
   Loaded Fluo-4_cache\masks\t84.npy, Fluo-4_cache\originals\t84.npy
   Loaded Fluo-4_cache\masks\t840.npy, Fluo-4_cache\originals\t840.npy
   Loaded 

   Loaded Fluo-4_cache\masks\t933.npy, Fluo-4_cache\originals\t933.npy
   Loaded Fluo-4_cache\masks\t934.npy, Fluo-4_cache\originals\t934.npy
   Loaded Fluo-4_cache\masks\t935.npy, Fluo-4_cache\originals\t935.npy
   Loaded Fluo-4_cache\masks\t936.npy, Fluo-4_cache\originals\t936.npy
   Loaded Fluo-4_cache\masks\t937.npy, Fluo-4_cache\originals\t937.npy
   Loaded Fluo-4_cache\masks\t938.npy, Fluo-4_cache\originals\t938.npy
   Loaded Fluo-4_cache\masks\t939.npy, Fluo-4_cache\originals\t939.npy
   Loaded Fluo-4_cache\masks\t94.npy, Fluo-4_cache\originals\t94.npy
   Loaded Fluo-4_cache\masks\t940.npy, Fluo-4_cache\originals\t940.npy
   Loaded Fluo-4_cache\masks\t941.npy, Fluo-4_cache\originals\t941.npy
   Loaded Fluo-4_cache\masks\t942.npy, Fluo-4_cache\originals\t942.npy
   Loaded Fluo-4_cache\masks\t943.npy, Fluo-4_cache\originals\t943.npy
   Loaded Fluo-4_cache\masks\t944.npy, Fluo-4_cache\originals\t944.npy
   Loaded Fluo-4_cache\masks\t945.npy, Fluo-4_cache\originals\t945.npy
   Loade

In [5]:
if __name__ == '__main__':
    main()

Using device: cuda:0


E:\Anaconda\envs\compvis\lib\site-packages\torch\nn\functional.py:2970: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn("Default upsampling behavior when mode={} is changed "


RuntimeError: CUDA out of memory. Tried to allocate 190.00 MiB (GPU 0; 6.00 GiB total capacity; 3.83 GiB already allocated; 58.45 MiB free; 3.85 GiB reserved in total by PyTorch)